# System rekomendacji na podstawie preferencji uzytkownika z dynamicznym obszarem wyszukiwania

In [2]:
import pandas as pd
import re

In [3]:
movies = pd.read_csv("../input/RS04_movies.csv")
ratings = pd.read_csv('../input/RS04_ratings.csv')

In [4]:
print(movies.shape)
print(ratings.shape)

(62423, 3)
(25000095, 4)


## Czyszczenie danych

In [6]:
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title) # usuwa wszystkie znaki które nie sa literami, cyframi ani spacjami 
    return title 

In [7]:
movies['clean_title']= movies['title'].apply(clean_title)

In [8]:
movies.sample(10)

,movieId,title,genres,clean_title
61192,205190,Mike Wallace Is Here (2019),Documentary,Mike Wallace Is Here 2019
37607,153022,Phantom Love (2007),Drama|Fantasy,Phantom Love 2007
24455,121503,Ator the Iron Warrior (1987),Action|Adventure|Drama|Fantasy|Romance,Ator the Iron Warrior 1987
36777,151121,The Big Cage (1933),(no genres listed),The Big Cage 1933
20688,106988,Carmina or Blow Up (Carmina o revienta) (2012),Comedy|Drama,Carmina or Blow Up Carmina o revienta 2012
56911,195201,Sad Hill Unearthed (2018),Documentary,Sad Hill Unearthed 2018
3431,3529,"Postman Always Rings Twice, The (1981)",Crime|Thriller,Postman Always Rings Twice The 1981
30392,136154,Bravissimo (1955),Comedy,Bravissimo 1955
53552,187661,Daddy (2017),Crime|Drama,Daddy 2017
21573,111150,Always a Bride (1953),Comedy,Always a Bride 1953


# Przekształcanie tekstu na macierz liczbową

TF-IDF to metoda przypisywania wartości liczbowej słowom w dokumencie na podstawie ich istotnosci w korpusie tekstu.
Term Frequency (TF) – częstość terminu
Inverse Document Frequency (IDF) – rzadkość terminu

- Słowa powszechne w korpusie tekstu mają niską wartość TF-IDF
- Słowa rzadkie maja wysoką warotść TF-IDF

- Unigramy (pojedyncze słowa, np. "Harry", "Potter")
- Bigramy (pary słów, np. "Harry Potter", "Potter i")

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer #przekszatałcanie tekstu na macierz liczbową

In [12]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) #uwzglednia zarówno pojedyncze słowa jak i pary słów

tfidf = vectorizer.fit_transform(movies["clean_title"]) # konwertuje kazdy tytuł na wektor TF-IDF

## Wyszukiwanie podobnych tytułów filmów

Podobieństwo cosinusowe jest miarą podobieństwa między dwoma niezerowymi wektorami zdefiniowanymi w przestrzeni iloczynu wewnętrznego, zawsze należy do przedziału [-1,1]

Mierzy, jak podobne są dwa wektory w przestrzeni wektorowej. Wartości wynikają z kąta między wektorami, gdzie:
Wartość 1 oznacza pełną zgodność.
Wartość 0 oznacza brak podobieństwa.
Wartość -1 oznacza całkowite przeciwieństwo.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity # miara podobienstwa kosinusowego
import numpy as np

In [16]:
def search(title):
    title = clean_title(title) # uzywanie funkcji oczyszczajacej
    query_vec = vectorizer.transform([title]) # zamiana tytułu na wektor TF-IDF
    similarity = cosine_similarity(query_vec, tfidf).flatten() # obliczny podobienstwo kosinusowe  zamienia wynik na wektor 1D(liste)
    indices = np.argpartition(similarity, -5)[-5:] #znajduje 5 najwyzszych indeksów które odpowiadaja najbardziej podobnym tytułom,tylko oostatnie 5 elementów
    results = movies.iloc[indices].iloc[::-1] # wybieramy wiersze na podstawie wcesniej obliczonych indeksów, odwracamy kolejnosc wyników
    
    return results

## Budowanie interaktywnego widgeta

In [18]:
import ipywidgets as widgets # biblioteka do tworzenia interaktywnych elementów 
from IPython.display import display # funkcja do dynamiczego wyswietlania wyników

In [19]:
movie_input = widgets.Text( # tworzenie pola tekstowego
    value='', # domyslna wartosc pola tekstowego
    description='Movie Title:', # opis pala tekstowego
    disabled=False
)
movie_list = widgets.Output() # widget do wyswietlania wyników

def on_type(data):
    with movie_list: # działanie na liscie movie_list 
        movie_list.clear_output() # czyszczenie poprzednich wyników
        title = data["new"] #
        if len(title) > 3: # jesli tekst ma wiecej niz 3 znaki 
            display(search(title)) # wyswiel wyniki z funkcji

movie_input.observe(on_type, names='value')


display(movie_input, movie_list) # wyswietla pole tekstowe i dynamiczny obasza w którym wyszukiwane sa filmy

Text(value='', description='Movie Title:')

Output()

## Tworzenie systemu rekomendacji

In [21]:
movie_id = 1 #identyfikator filmu
movie = movies[movies["movieId"] == movie_id]
movie

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995


In [22]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique() # wszystkie oceny i ocene wieksza nic 4 i zwracamy liste unikalnych identyfikatorów
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"] # wybieramy oceny wystawione przez uzytkowników i filmy z ocena wieksze niz 4 i zwracamy liste identyfikatorów

In [23]:
ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]

,userId,movieId,rating,timestamp
5101,36,1,5.0,857131378
5105,36,34,5.0,834413787
5111,36,110,5.0,834412999
5114,36,150,5.0,839928587
5127,36,260,5.0,857131062
...,...,...,...,...
24998854,162533,60069,4.5,1280919889
24998861,162533,67997,4.5,1280920712
24998876,162533,78499,4.5,1281405901
24998884,162533,81591,4.5,1297289876


In [24]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users) #zliczamy ile razy kazdy film pojawia sie i dizleimy przez liczbe uzytkowników
similar_user_recs = similar_user_recs[similar_user_recs > .10] # zachowujemy tylko te filmy które były wysoko oceniane przez wiecej niz 10% uzytkowników

In [25]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)] # wszystkie oceny dotyczące tych filmów i oceny wyzsze niz 4
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique()) # zlicza jak czesto film pojawia sie w ocenach uzytkownika dzielimy przez liczbe uzytkowników

In [26]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1) # łaczenie w jeden dataframe
rec_percentages.columns = ["similar", "all"] #nadanie nazw kolumnom

Mierzymy jak bardzo dany film jest specyficzny dla uzytkowników podobnych do nasw porównaniu do ogolnej społecznosci.

Jeśli score > 1, oznacza to, że film jest znacznie bardziej popularny wśród użytkowników podobnych do nas niż w całej społeczności → dobry kandydat na rekomendację.
Jeśli score ≈ 1, film jest tak samo popularny wśród podobnych użytkowników i ogółu.
Jeśli score < 1, film jest mniej popularny wśród podobnych użytkowników niż w ogóle, więc może nie być najlepszą rekomendacją.

In [28]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"] #
rec_percentages = rec_percentages.sort_values("score", ascending=False) # sortowanie od najwiekszej wartosci score


In [29]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId") # wyubiramy 10 filmów, łaczymy z pełnymi informacjami o filmach, 

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


## Tworzenie ogólnej funkcji rekomentdacji

In [31]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique() # filtrujemy uzytkowników który ocenili dany film na wiecej niz 4
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]] #wyswietlamy tylko kolumny: "score", "title", "genres"

## Tworzenie interaktywnego widgetu rekomendacji

In [33]:
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output() # obszar wyswietlania rekomendacji

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()  # Usuwamy poprzednie wyniki
        title = data["new"] # Pobieramy nowy wpisany tytuł
        if len(title) > 4: # Sprawdzamy, czy użytkownik wpisał co najmniej 5 znaków
            results = search(title) # Szukamy filmu w bazie
            movie_id = results.iloc[0]["movieId"]  # Pobieramy `movieId` pierwszego znalezionego filmu
            display(find_similar_movies(movie_id)) # Wyświetlamy listę podobnych filmów

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()